In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'C:\Users\shaif\Downloads\Compressed\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in os.listdir(directory_path):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [2]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
Y_train_encoded = to_categorical(Y_train)
indices = np.arange(len(X_train))
np.random.shuffle(indices)
X_train_shuffled = X_train[indices]
Y_train_encoded_shuffled = Y_train_encoded[indices]
X_train, X_test, y_train, y_test = train_test_split(X_train_shuffled, Y_train_encoded_shuffled, test_size=0.2, random_state=42)

In [ ]:
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

model_path = r"C:\Users\shaif\Downloads\invasive_ssl1.h5"
model = load_model(model_path)
#model.layers[0].trainable = False
x = model.layers[-8].output  
output = Dense(5, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=40, mode='max', verbose=1)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = new_model.fit(X_train, y_train, batch_size=32, epochs=150,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/150
985/985 [==============================] - 66s 56ms/step - loss: 31.7675 - accuracy: 0.4299 - val_loss: 2.1137 - val_accuracy: 0.5473
Epoch 2/150
985/985 [==============================] - 49s 49ms/step - loss: 2.8954 - accuracy: 0.4932 - val_loss: 2.0357 - val_accuracy: 0.4789
Epoch 3/150
985/985 [==============================] - 60s 60ms/step - loss: 1.9568 - accuracy: 0.5349 - val_loss: 1.7262 - val_accuracy: 0.5600
Epoch 4/150
985/985 [==============================] - 56s 57ms/step - loss: 2.0034 - accuracy: 0.5670 - val_loss: 1.7217 - val_accuracy: 0.6227
Epoch 5/150
985/985 [==============================] - 57s 58ms/step - loss: 1.6519 - accuracy: 0.5866 - val_loss: 2.4593 - val_accuracy: 0.6280
Epoch 6/150
985/985 [==============================] - 52s 53ms/step - loss: 1.3193 - accuracy: 0.6143 - val_loss: 1.2106 - val_accuracy: 0.6558
Epoch 7/150
985/985 [==============================] - 60s 61ms/step - loss: 1.3260 - accuracy: 0.6234 - val_loss: 1.1780 - val_a

985/985 [==============================] - 54s 55ms/step - loss: 0.6507 - accuracy: 0.7893 - val_loss: 114051.7969 - val_accuracy: 0.7315
Epoch 57/150
985/985 [==============================] - 49s 50ms/step - loss: 0.5726 - accuracy: 0.7927 - val_loss: 0.5168 - val_accuracy: 0.8008
Epoch 58/150
725/985 [=====================>........] - ETA: 12s - loss: 0.5271 - accuracy: 0.7972

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

In [ ]:
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(include_top=False, weights="imagenet", input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x)
ne_model = Model(inputs=base_model.input, outputs=output)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, mode='max', verbose=1)
ne_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = ne_model.fit(X_train, y_train, batch_size=32, epochs=150,validation_split=0.2, callbacks=[early_stopping])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = ne_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x) 
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)